### **title 기반 Search Hit Rate (검색 성공률) 설계**

- **Goal**: GPT가 생성한 "가수 - 제목"이 실제로 Spotify에 존재하는지 검증하여, **할루시네이션(거짓 정보)**을 방지한다. (Spotify search API사용해서 실제로 검색하기)

- **Metric**:evaluation_result_semantic_final.xlsx (GPT가 추천한 결과 파일)를 읽어서, Spotify에 하나씩 검색해 보고 성공률을 계산

입력: evaluation_result_semantic_final.xlsx (GPT 추천 결과)

작업: Spotify Search API 호출 (artist + track 조합 검색)

출력: search_hit_rate_result.xlsx (검색 결과 및 Spotify URL 포함)

- **KPI**: 검색성공률((Spotify API 검색 결과가 1건 이상인 추천 수 / 전체 추천 시도 수) × 100) 95%이상

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# ===========================================================
# 1. 설정 (Spotify API Key 입력)
# ===========================================================
SPOTIFY_CLIENT_ID = ""
SPOTIFY_CLIENT_SECRET = ""

INPUT_FILE = "evaluation_result_semantic_final.xlsx"  # 아까 GPT가 만든 파일
OUTPUT_FILE = "search_hit_rate_result.xlsx"           # 결과 저장할 파일

# ===========================================================
# 2. Spotify 클라이언트 연결 (인증)
# ===========================================================
# User Login 필요 없는 'Client Credentials Flow' 사용 (검색용)
try:
    auth_manager = SpotifyClientCredentials(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET
    )
    sp = spotipy.Spotify(auth_manager=auth_manager)
    print(" Spotify API 연결 성공!")
except Exception as e:
    print(f" Spotify 연결 실패: {e}")
    exit()

# ===========================================================
# 3. 메인 로직: 검색 성공률 테스트
# ===========================================================
print(f"\n Search Hit Rate 측정을 시작합니다... (파일: {INPUT_FILE})")

try:
    df = pd.read_excel(INPUT_FILE)
except FileNotFoundError:
    print(" 엑셀 파일이 없어요! 이전 단계(의미적 일치성) 코드를 먼저 돌려주세요.")
    exit()

results = []
hit_count = 0
total_count = 0

for index, row in df.iterrows():
    gpt_artist = row.get('Recommended Artist', 'Unknown')
    gpt_title = row.get('Recommended Title', 'Unknown')
    
    # 에러난 케이스는 건너뜀
    if gpt_artist == "Unknown" or gpt_title == "Unknown":
        print(f" Case {row['Case ID']}: GPT 생성 실패로 스킵")
        continue

    total_count += 1
    print(f" Searching: {gpt_artist} - {gpt_title} ...", end=" ")
    
    try:
        # [핵심] Spotify Search API 호출
        # 정확도를 위해 "artist:가수 track:제목" 형태로 검색 쿼리 구성
        query = f"artist:{gpt_artist} track:{gpt_title}"
        search_result = sp.search(q=query, type='track', limit=1)
        
        items = search_result['tracks']['items']
        
        if len(items) > 0:
            #  HIT! (검색 성공)
            track = items[0]
            print("✅ HIT!")
            hit_count += 1
            
            results.append({
                **row, # 기존 데이터 유지
                "Spotify Search": "SUCCESS",
                "Spotify ID": track['id'],
                "Spotify URL": track['external_urls']['spotify'],
                "Real Artist": track['artists'][0]['name'], # 실제 스포티파이 표기
                "Real Title": track['name']
            })
        else:
            #  MISS! (없는 노래 / 할루시네이션)
            print("❌ MISS (Not Found)")
            results.append({
                **row,
                "Spotify Search": "FAIL",
                "Spotify ID": None,
                "Spotify URL": None,
                "Real Artist": None,
                "Real Title": None
            })
            
    except Exception as e:
        print(f" Error: {e}")
        results.append({**row, "Spotify Search": "ERROR"})
    
    # API 도배 방지
    time.sleep(0.2)

# ===========================================================
# 4. 결과 요약 및 저장
# ===========================================================
result_df = pd.DataFrame(results)
result_df.to_excel(OUTPUT_FILE, index=False)

hit_rate = (hit_count / total_count) * 100 if total_count > 0 else 0

print("\n" + "="*50)
print(f" 검증 완료! 파일 저장됨: {OUTPUT_FILE}")
print(f" [KPI] Search Hit Rate (검색 성공률): {hit_rate:.2f}%")
print(f"   - 시도: {total_count}건")
print(f"   - 성공: {hit_count}건")
print(f"   - 실패: {total_count - hit_count}건")

if hit_rate >= 95:
    print(" 합격! GPT가 실존하는 노래를 아주 잘 추천합니다.")
else:
    print(" 주의! 없는 노래를 지어내는 비율이 높습니다. 프롬프트 개선이 필요합니다.")
print("="*50)

 Spotify API 연결 성공!

 Search Hit Rate 측정을 시작합니다... (파일: evaluation_result_semantic_final.xlsx)
 Searching: Ludovico Einaudi - Nuvole Bianche ... ✅ HIT!
 Searching: AC/DC - Thunderstruck ... ✅ HIT!
 Searching: Max Richter - Dream 3 (In the Midst of My Life) ... ✅ HIT!
 Searching: Miles Davis - So What ... ✅ HIT!
 Searching: NewJeans - Hype Boy ... ✅ HIT!
 Searching: Lo-fi Beats - Chill Study Beats ... ✅ HIT!
 Searching: Dua Lipa - Levitating ... ✅ HIT!
 Searching: IU - Through the Night ... ✅ HIT!
 Searching: Metallica - Nothing Else Matters ... ✅ HIT!
 Searching: Ludwig van Beethoven - Symphony No. 5 in C Minor, Op. 67: I. Allegro con brio ... ✅ HIT!
 Searching: J. Cole - Apparently ... ✅ HIT!
 Searching: BTS - Euphoria ... ✅ HIT!
 Searching: ODESZA - A Moment Apart ... ✅ HIT!
 Searching: Foo Fighters - Times Like These ... ✅ HIT!
 Searching: Johann Sebastian Bach - Goldberg Variations, BWV 988: Aria ... ✅ HIT!
 Searching: Rage Against The Machine - Killing In The Name ... ✅ HIT!
 Sear

Brian Eno - Ambient 1: Music for Airports 는 노래(Track)가 아니기  때문에 실패(Miss), (스포티파이청취이력분석을 통해 입력된 선호임). 하지만 노래(트랙)에는 해당 검색결과가 없음-> 할루시네이션은 아니지만 음악추천 관점에서는 노래가 아니기 때문에 도메인 위반임 -> 프롬프트 수정하기

>검색 성공률(Search Hit Rate) 테스트 결과, 20건 중 19건 성공(95%). 실패한 1건은 >GPT가 음악 대신 팟캐스트를 추천한 사례로, 이는 프롬프트 제약조건 강화로 해결 가능함